# Inspect saved experiment datapoints and activations
This notebook loads a saved experiment pickle (matching `math500*`) and prints summaries of datapoint text fields and captured activations. It is written to be robust to missing attributes and different activation tensor types.

In [2]:
import os, glob, pickle, sys
from pprint import pprint

# Optional: prefer torch if available for tensor shape introspection
try:
    import torch
    TORCH_AVAILABLE = True
except Exception:
    TORCH_AVAILABLE = False

try:
    import numpy as np
    NUMPY_AVAILABLE = True
except Exception:
    NUMPY_AVAILABLE = False

def _shape_of_item(x):
    if x is None:
        return None
    if TORCH_AVAILABLE and isinstance(x, torch.Tensor):
        return tuple(x.shape)
    if NUMPY_AVAILABLE and isinstance(x, np.ndarray):
        return x.shape
    # common Python containers with first element shapes
    if isinstance(x, (list, tuple)) and len(x) > 0:
        # attempt to describe first few entries
        shapes = []
        for v in x[:3]:
            shapes.append(_shape_of_item(v))
        return shapes
    return type(x).__name__

def summarize_activation_dict(act_dict, max_preview=3):
    if act_dict is None:
        print('    <None>')
        return 0
    if not isinstance(act_dict, dict):
        print('    <not-a-dict>', type(act_dict))
        return 0
    total = 0
    for k, lst in act_dict.items():
        if lst is None:
            print(f'    - {k}: None')
            continue
        try:
            length = len(lst)
        except Exception:
            print(f'    - {k}: <not-a-list> {type(lst)}')
            continue
        total += length
        preview = []
        seen = 0
        for item in lst:
            if item is None:
                preview.append(None)
            else:
                preview.append(_shape_of_item(item))
            seen += 1
            if seen >= max_preview:
                break
        print(f'    - {k}: {length} entries, preview shapes/types: {preview}')
    return total

In [9]:
# Minimal loader: load the latest `math500*.pkl` and expect a list of datapoints
import glob
import os

# Hide GPUs from this Python process so deserialization can't allocate to CUDA
os.environ.setdefault('CUDA_VISIBLE_DEVICES', '')

repo_root = os.path.abspath(os.getcwd())
path = os.path.join(repo_root, 'math500_no_injection_20260119_174803.pkl')

obj = None
# Prefer torch.load with map_location='cpu' and weights_only=False to avoid allocating tensors on CUDA
try:
    import torch
    try:
        obj = torch.load(path, map_location='cpu', weights_only=False)
        print('Loaded with torch.load(map_location="cpu", weights_only=False)')
    except Exception as e:
        print('torch.load failed, falling back to pickle.load:', e)
        with open(path, 'rb') as f:
            obj = pickle.load(f)
except Exception as e:
    # torch not available or import failed; fallback to pickle
    print('torch unavailable or errored, using pickle.load:', e)
    with open(path, 'rb') as f:
        obj = pickle.load(f)

# Prefer a plain list of datapoints; if the pickle contains an Experiment-like
# object with a `.datapoints` attribute, extract that list.
if isinstance(obj, list):
    datapoints = obj
    print('Loaded list of datapoints, count =', len(datapoints))
elif hasattr(obj, 'datapoints'):
    datapoints = obj.datapoints
    print('Loaded object with .datapoints, count =', len(datapoints))
else:
    raise TypeError('Pickle did not contain a list of datapoints or an object with `.datapoints`')

# Expose datapoints in the notebook namespace
globals()['datapoints'] = datapoints

torch.load failed, falling back to pickle.load: Invalid magic number; corrupt file?
Loaded list of datapoints, count = 10


In [12]:
print(datapoints[0].activations_upto_injection.keys())

dict_keys(['layer_0_attention', 'layer_1_attention', 'layer_2_attention', 'layer_3_attention', 'layer_4_attention', 'layer_5_attention', 'layer_6_attention', 'layer_7_attention', 'layer_8_attention', 'layer_9_attention', 'layer_10_attention', 'layer_11_attention', 'layer_12_attention', 'layer_13_attention', 'layer_14_attention', 'layer_15_attention', 'layer_16_attention', 'layer_17_attention', 'layer_18_attention', 'layer_19_attention', 'layer_20_attention', 'layer_21_attention', 'layer_22_attention', 'layer_23_attention', 'layer_24_attention', 'layer_25_attention', 'layer_26_attention', 'layer_27_attention'])


In [ ]:
layer_1_attention = datapoints[0].question_contents["layer_1_attention"]

False